# Design Patterns with LangGraph

## 🧠 Introduction to LangGraph

**LangGraph** is a powerful framework built on top of **LangChain** that helps developers design **stateful, multi-step AI applications** using **graph-based workflows**.

Instead of writing sequential chains or complex agents manually, LangGraph allows us to represent LLM logic as a **graph of nodes** — where:
- Each **node** represents a computation or decision step.
- **Edges** define how data flows between nodes.
- The entire graph maintains **state** throughout the conversation.

Think of LangGraph as:
> "A framework that brings deterministic control to agentic workflows powered by LLMs."

It blends the flexibility of traditional agents with the **reliability and observability** of a flow-based architecture.


====================================================================================================================================================================================
====================================================================================================================================================================================

## 💡 LangGraph – Theoretical Examples

### 🧭 Example 1: Simple RAG Workflow
A basic Retrieval-Augmented Generation (RAG) graph can have three nodes:

1. **Query Analyzer Node** – decides if the question needs retrieval.  
2. **Retriever Node** – fetches documents from the vector store.  
3. **Answer Generator Node** – uses the LLM to synthesize an answer.

**Flow:**  
User Query ➜ Query Analyzer ➜ Retriever ➜ Answer Generator ➜ Response

This gives fine control: if a query doesn’t need retrieval, the graph can skip the retriever step.

---

### 🧩 Example 2: FAQ Chatbot with Guardrails
For a PDF FAQ bot:
- **Input Node:** takes user question.  
- **Retriever Node:** fetches relevant chunks from the document.  
- **Answer Node:** LLM generates a summary answer.  
- **Verifier Node:** double-checks if the answer is grounded in source documents.  
- **Fallback Node:** returns “I’m not sure” if confidence < threshold.

This ensures reliability and factual accuracy.
---

✅ **Key takeaway**
LangGraph = *Visual + Deterministic + Stateful AI Flow Control.*

You can visualize it as:

====================================================================================================================================================================================
====================================================================================================================================================================================

In [ ]:
# Multi agent workflow with LangGraph, where a supervisor dynamically routes user queries to specialized agents like Weather, 
# Flights, and News. 


# Imports
# Define LLM
# Router Model

# Agent
# SuperVisor Agent
# Weather Agent
# News Agent

# Graph Construction
# Invoke Graph 

In [ ]:
# Multi agent workflow with LangGraph, where a supervisor dynamically routes user queries to specialized agents like Weather, 
# Flights, and News. 
# 
###############################################################
# SETUP: IMPORTS & ENVIRONMENT
###############################################################
import operator
from typing import Annotated, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langgraph.types import Command

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv

load_success = load_dotenv()


###############################################################
# LLM INITIALIZATION
###############################################################
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)


###############################################################
# UPDATED ROUTER MODEL — ADD "News"
###############################################################
class Router(BaseModel):
    """
    Decides the next agent to route to.
    Now supports: Weather / Flights / News / __end__
    """
    next_agent: Literal["Weather", "Flights", "News", "__end__"]

supervisor_model = llm.with_structured_output(Router)


###############################################################
# SUPERVISOR FUNCTION
###############################################################
def supervisor(state: MessagesState) -> Command:
    print("--- 🧑‍💼 SUPERVISOR ---")

    prompt = f"""
    You are a supervisor routing tasks to a specialist agent.
    Based on the user's request, choose the appropriate agent.

    Agents:
    - Weather: For weather forecast questions.
    - Flights: For flight information.
    - News: For news or headlines.
    
    If the user is finishing the conversation, choose '__end__'.

    User message: "{state['messages'][-1].content}"
    """

    if isinstance(state['messages'][-1], HumanMessage):
        response = supervisor_model.invoke(prompt)
        print(f"Supervisor routing to: {response.next_agent}")
        return Command(goto=response.next_agent)
    else:
        return Command(goto="__end__")


###############################################################
# WEATHER AGENT
###############################################################
def weather_agent(state: MessagesState) -> Command:
    print("--- ☀️ WEATHER AGENT ---")

    prompt = f"""
    You are a weather forecaster.
    Provide a short mock weather forecast for the user's location.

    User message: "{state['messages'][-1].content}"
    """

    response = llm.invoke(prompt)
    print(f"Response: {response.content}")

    return Command(
        goto="supervisor",
        update={"messages": [response]},
    )


###############################################################
# FLIGHTS AGENT
###############################################################
def flights_agent(state: MessagesState) -> Command:
    print("--- ✈️ FLIGHTS AGENT ---")

    prompt = f"""
    You are a flight information assistant.
    Give a short mock flight detail for user's requested destination.

    User message: "{state['messages'][-1].content}"
    """

    response = llm.invoke(prompt)
    print(f"Response: {response.content}")

    return Command(
        goto="supervisor",
        update={"messages": [response]},
    )


###############################################################
# ⭐ NEW NEWS AGENT
###############################################################
def news_agent(state: MessagesState) -> Command:
    print("--- 📰 NEWS AGENT ---")

    prompt = f"""
    You are a news assistant.
    Provide 2–3 mock breaking news headlines relevant to the user's message.

    User message: "{state['messages'][-1].content}"
    """

    response = llm.invoke(prompt)
    print(f"Response: {response.content}")

    return Command(
        goto="supervisor",
        update={"messages": [response]},
    )


###############################################################
# GRAPH CONSTRUCTION
###############################################################
builder = StateGraph(MessagesState)

builder.add_node("supervisor", supervisor)
builder.add_node("Weather", weather_agent)
builder.add_node("Flights", flights_agent)
builder.add_node("News", news_agent)  # <-- NEW NODE REGISTERED

builder.add_edge(START, "supervisor")

graph = builder.compile()


In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # The graph.stream() method invokes the graph and streams the results.
#     What is the weather in Chennai tomorrow?
    # Find me flights to Bangalore
    events = graph.stream({"messages":[HumanMessage(content=user_input)]})
    for event in events:
        # We only print the AI's responses to the user.
        if "messages" in event:
            event["messages"][-1].pretty_print()

--- 🧑‍💼 SUPERVISOR ---
----
[HumanMessage(content='What will be the weather like in Chennai tomorrow morning?', additional_kwargs={}, response_metadata={}, id='e9960eb4-b5b0-46e0-ab9d-bbc30c1898d1')]
----
Supervisor routing to: Weather
--- ☀️ WEATHER AGENT ---
Response: Good morning! Here's your forecast for Chennai tomorrow morning:

Expect partly cloudy skies with a chance of light drizzle. Temperatures will be around 28 degrees Celsius. The humidity will be high, making it feel a little warmer. Winds will be light and variable. Have a great morning!
--- 🧑‍💼 SUPERVISOR ---
----
[HumanMessage(content='What will be the weather like in Chennai tomorrow morning?', additional_kwargs={}, response_metadata={}, id='e9960eb4-b5b0-46e0-ab9d-bbc30c1898d1'), AIMessage(content="Good morning! Here's your forecast for Chennai tomorrow morning:\n\nExpect partly cloudy skies with a chance of light drizzle. Temperatures will be around 28 degrees Celsius. The humidity will be high, making it feel a lit

In [ ]:
# Multi agent workflow with LangGraph, where a supervisor dynamically routes user queries to specialized agents like Weather, 
# Flights, and News. 


# Imports
# Define LLM
# Router Model

# generator
# evaluator
# decide

# Graph Construction
# Invoke Graph 

In [ ]:
# Workflow with three nodes - generator, evaluator, and finalize
# This iteratively improves an LLM-generated draft based on feedback
# It loops between generation and evaluation until the evaluator returns “APPROVED,” then outputs the final draft.

import operator
from typing import Annotated, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langgraph.types import Command

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv

load_success = load_dotenv()

# LLM client
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)

# State
class State(TypedDict):
    task: str
    draft: str
    feedback: str
    final: str

# Nodes
def generator(state: State):
    """Generate an initial or refined draft."""
    prompt = f"""
You are an assistant helping to complete the following task:

Task:
{state['task']}

Current Draft:
{state.get('draft', 'None')}

Feedback:
{state.get('feedback', 'None')}

Instructions:
- If there is no draft and no feedback, generate a clear and complete response to the task.
- If there is a draft but no feedback, improve the draft as needed for clarity and quality.
- If there is both a draft and feedback, revise the draft by incorporating the feedback directly.
- Always produce a single, improved draft as your output.
"""
    resp = llm.invoke(prompt)
    return {"draft": resp.content.strip()}

def evaluator(state: State):
    """Evaluate the draft and give feedback or approval."""
    prompt = f"""Evaluate the following draft, based on the given task.
If it meets the requirements, reply exactly 'APPROVED'.
Otherwise, provide constructive feedback for improvement.
Task:
{state['task']}
Draft:
{state['draft']}"""
    resp = llm.invoke(prompt)
    print(f"""
================= DRAFT =================
{state['draft']}

================ FEEDBACK ===============
{resp.content.strip()}
========================================
""")
    return {"feedback": resp.content.strip()}

def decide(state: State) -> str:
    """Decide next step: either approve and finish, or refine again."""
    if "APPROVED" in state["feedback"].upper():
        return "approved"
    return "refine"

def finalize(state: State):
    """Save the final approved draft."""
    return {"final": state["draft"]}

# Build the graph
builder = StateGraph(State)

builder.add_node("generator", generator)
builder.add_node("evaluator", evaluator)
builder.add_node("finalize", finalize)

builder.add_edge(START, "generator")
builder.add_edge("generator", "evaluator")
builder.add_edge("evaluator", "finalize")

# Conditional edges from decide
builder.add_conditional_edges(
    "evaluator",
    decide,
    {
        "approved": "finalize",   # stop loop
        "refine": "generator",    # go back for improvement
    },
)

builder.add_edge("finalize", END)

graph = builder.compile()

In [8]:
# Run example
# You have six horses and want to race them to see which is fastest. What is the best way to do this?
# Write a clear and concise introduction for a blog post explaining what LangGraph is and why developers should use it.
# Summarize the key challenges in scaling a Node.js application to support 100k concurrent users.

input_task = "Summarize the key challenges in scaling a Node.js application to support 100k concurrent users."
result = graph.invoke({"task": input_task})

print("\nFinal Answer:\n", result["final"])


================= DRAFT =================
Scaling a Node.js application to handle 100,000 concurrent users presents several key challenges. These can be broadly categorized into infrastructure, application code, database, and monitoring/operations.

**1. Infrastructure Limitations:**

*   **Single-Threaded Nature of Node.js:** Node.js, while event-driven and non-blocking, primarily operates on a single thread. This can become a bottleneck when handling CPU-intensive tasks. Techniques like clustering (using Node.js's `cluster` module or process managers like PM2) or offloading CPU-intensive tasks to worker threads are crucial.
*   **Server Capacity:** A single server likely won't be sufficient. Load balancing across multiple servers is essential to distribute the load and prevent overload. This requires choosing a suitable load balancer (e.g., Nginx, HAProxy, cloud provider load balancers).
*   **Network Bandwidth:** Sufficient network bandwidth is critical to handle the increased traf